# Live demo: Processing gravity data with Fatiando a Terra

This notebook is based on the [Fatiando tutorial for processing gravity data](https://github.com/fatiando/tutorials).

## Import packages

In [ ]:
import pygmt
import pyproj
import numpy as np
import pandas as pd
import xarray as xr

import ensaio
import verde as vd
import boule as bl
import harmonica as hm

## Load gravity data for Bushveld Igenous Complex (Southern Africa) and a DEM

We can use [Ensaio](https://www.fatiando.org/ensaio) to download our sample data files.
Let's download some gravity data of the Bushveld Igenous Complex, Southern Africa:

And use Pandas for reading the csv file

In [ ]:
data = pd.read_csv(fname).drop(columns=["gravity_disturbance_mgal", "gravity_bouguer_mgal"])
data

In [ ]:
# Obtain the region to plot using Verde ([W, E, S, N])
region_deg = vd.get_region((data.longitude, data.latitude))

fig = pygmt.Figure()
fig.basemap(projection="M15c", region=region_deg, frame=True)
pygmt.makecpt(cmap="viridis", series=[data.gravity_mgal.min(), data.gravity_mgal.max()])
fig.plot(
    x=data.longitude,
    y=data.latitude,
    color=data.gravity_mgal,
    cmap=True,
    style="c0.1c",
)
fig.colorbar(frame='af+l"Observed Gravity [mGal]"')
fig.show()

Let's download a DEM for Southern Africa:

And use Xarray to load the netCDF file:

In [ ]:
topography = xr.load_dataarray(fname)
topography

### Crop the topography around Bushveld Igeneous complex area

In [ ]:
# Plot topography using pygmt
fig = pygmt.Figure()
topo_region = vd.get_region((topography.longitude.values, topography.latitude.values))
fig.basemap(projection="M15c", region=topo_region, frame=True)

vmin, vmax = topography.values.min(), topography.values.max()
pygmt.makecpt(cmap="etopo1", series=[vmin, vmax])
fig.grdimage(topography)

fig.colorbar(frame='af+l"Topography [m]"')
fig.show()

## Compute gravity disturbance

We can use [Boule](https://www.fatiando.org/boule) to compute the normal gravity of the WGS84 reference ellipsoid on any point.

And compute the gravity disturbance as the difference between the observed gravity and the normal gravity:

In [ ]:
data["gravity_disturbance_mgal"] = data.gravity_mgal - normal_gravity
data

In [ ]:
fig = pygmt.Figure()
fig.basemap(projection="M15c", region=region_deg, frame=True)

maxabs = vd.maxabs(data.gravity_disturbance_mgal)
pygmt.makecpt(cmap="polar", series=[-maxabs, maxabs])
fig.plot(
    x=data.longitude,
    y=data.latitude,
    color=data.gravity_disturbance_mgal,
    cmap=True,
    style="c4p",
)

fig.colorbar(frame='af+l"Gravity disturbance [mGal]"')
fig.show()

## Remove terrain effect

We can use [Harmonica](https://www.fatiando.org/harmonica) for forward modelling the gravitational effect of the terrain through rectangular prisms.
To do so we need to project the coordinates of the data and the DEM to plain coordinates.

### Project the data to plain coordinates

Define the Mercator projeciton using `pyproj`:

In [ ]:
projection = pyproj.Proj(proj="merc", lat_ts=data.latitude.mean())

In [ ]:
easting, northing = projection(data.longitude.values, data.latitude.values)

In [ ]:
data["easting"] = easting
data["northing"] = northing
data

### Project the topography to plain coordinates

### Compute gravitational effect of the layer of prisms

Create a model of the terrain with prisms


![](nb-images/prisms.svg)

Calculate the gravitational effect of the terrain

In [ ]:
coordinates = (data.easting, data.northing, data.height_geometric_m)

Calculate the Bouguer disturbance

In [ ]:
data["bouguer_mgal"] = data.gravity_disturbance_mgal - terrain_effect
data

In [ ]:
fig = pygmt.Figure()
fig.basemap(projection="M15c", region=region_deg, frame=True)

maxabs = vd.maxabs(data.bouguer_mgal)
pygmt.makecpt(cmap="polar", series=[-maxabs, maxabs])
fig.plot(
    x=data.longitude,
    y=data.latitude,
    color=data.bouguer_mgal,
    cmap=True,
    style="c4p",
)
fig.colorbar(frame='af+l"Bouguer gravity disturbance [mGal]"')
fig.show()

## Calculate residuals

We can calculate a regional field by defining deep equivalent sources with [Harmonica](https://www.fatiando.org/harmonica):

![](nb-images/equivalent-sources.svg)

In [ ]:
data["residuals_mgal"] = residuals
data

In [ ]:
fig = pygmt.Figure()
fig.basemap(projection="M15c", region=region_deg, frame=True)

maxabs = np.quantile(np.abs(data.residuals_mgal), 0.99)
pygmt.makecpt(cmap="polar", series=[-maxabs, maxabs])
fig.plot(
    x=data.longitude,
    y=data.latitude,
    color=data.residuals_mgal,
    cmap=True,
    style="c5p",
)
fig.colorbar(frame='af+l"Residuals [mGal]"')
fig.show()

## Grid the residuals with Equivalent Sources

We can use [Harmonica](https://www.fatiando.org/harmonica) to grid the residuals though the equivalent sources technique

Define coordiantes of the grid

In [ ]:
fig = pygmt.Figure()
fig.basemap(projection="M15c", region=region_deg, frame=True)

scale = np.quantile(np.abs(grid.residuals), 0.995)
pygmt.makecpt(cmap="polar", series=[-scale, scale], no_bg=True)
fig.grdimage(
    grid.residuals,
    shading="+a45+nt0.15",
    cmap=True,
)
fig.colorbar(frame='af+l"Residuals [mGal]"')
fig.show()

![Bushveld geologic map](nb-images/bushveld_igneous_complex_geology.jpg)

Geology and mineral exploration sites of the Bushveld Igneous Complex ([public domain](https://en.wikipedia.org/wiki/File:Bushveld_Igneous_Complex.png)). 